In [1]:
using LinearAlgebra

In [2]:
mutable struct Tableau
    A::Array{Float64,2}
    b::Array{Float64,1}
    c::Array{Float64,1}
    basis::Dict{Int64,Int64}
end

In [3]:
mutable struct TableauI
    A::Array{Float64,2}
    b::Array{Float64,1}
    c::Array{Float64,1}
    cw::Array{Float64, 1}
    basis::Dict{Int64,Int64}
end

In [4]:
mutable struct Solution
    F::Float64
    Decision::Array{Float64, 1}
    Slack::Array{Float64, 1}
end

In [5]:
function obtain_solution(T::Tableau, 
                        decision_variables::Array{Int64, 1},
                        slack_variables::Array{Int64, 1}, 
                        artificial_variables::Array{Int64, 1})
    
    #array for decision variables
    X = zeros((size(decision_variables, 1)))
    #array for slack variables
    X_hat = zeros((size(slack_variables)))
    
    #for each line of A, look at variable at basis
    for (key, value) in T.basis
        if value in slack_variables
            X_hat[(value - size(X, 1))] = T.b[key]
        else
            X[value] = T.b[key]
            
        end
    end
    return Solution(-1*T.b[size(T.b, 1)],
                    X,
                    X_hat)
end

obtain_solution (generic function with 1 method)

In [6]:
function pivot_tableauI(T::TableauI, r::Int64, s::Int64)
    """
    Function that pivot simplex table on the coefficient [r, s]
    Return pivoted table
    """
    
    #remove line from basis
    delete!(T.basis, r)
    push!(T.basis, r => s)   
    
    
    #Pivot A, b, c with new non-zero variables
    T.b[(size(T.b, 1) - 1)] = T.b[(size(T.b, 1) - 1)] - T.c[s]*T.b[r]/T.A[r, s]
    T.b[(size(T.b, 1))] = T.b[(size(T.b, 1))] - T.cw[s]*T.b[r]/T.A[r, s]
    T.c .= T.c .- (T.c[s] .* T.A[r, :]/T.A[r, s])
    T.cw .= T.cw .- (T.cw[s] .* T.A[r, :]/T.A[r, s])
    
    aux = T.b[r]/T.A[r, s]

    T.b[1:(size(T.b, 1) - 2)] .= T.b[1:(size(T.b, 1) - 2)] .- T.A[:, s]*aux
    T.b[r] = aux
    
    T.A[r, :] .= T.A[r, :]/T.A[r, s]
    for i = 1:size(T.A, 1)
        if i != r
            T.A[i, :] .= T.A[i, :] .- (T.A[i, s] * T.A[r, :])
        end
    end
    return T
end


function simplex_iterationI(T::TableauI)
    
    k = 0
    #Can't increase z
    while (maximum(T.cw) > 0) & (k < 100)
        #Column to pivot
        s = argmax(T.cw)
        
        #Check if is unbounded
        if maximum(T.A[:, s]) <= 0
            error("Problem unbounded.")
        end

        #Ratio test to chose row to pivot
        r = 1
        min_ratio = T.b[1]/T.A[1, s]
        for i = 2:size(T.A, 1)
            if T.A[i, s] > 0
                new_ratio = T.b[i]/T.A[i, s]
                if new_ratio < min_ratio
                    r = i
                    min_ratio = new_ratio
                end
            end
        end
        
        T = pivot_tableauI(T, r, s) 
        k = k +1
    end
    return T
end

simplex_iterationI (generic function with 1 method)

In [7]:
function pivot_tableau(T::Tableau, r::Int64, s::Int64)
    """
    Function that pivot simplex table on the coefficient [r, s]
    Return pivoted table
    """
    
    #remove line from basis
    delete!(T.basis, r)
    push!(T.basis, r => s)   
    
    
    #Pivot A, b, c with new non-zero variables
    T.b[size(T.b, 1)] = T.b[size(T.b, 1)] - T.c[s]*T.b[r]/T.A[r, s]
    T.c .= T.c .- (T.c[s] .* T.A[r, :]/T.A[r, s])
    
    aux = T.b[r]/T.A[r, s]

    T.b[1:(size(T.b, 1) - 1)] .= T.b[1:(size(T.b, 1) - 1)] .- T.A[:, s]*aux
    T.b[r] = aux
    
    T.A[r, :] .= T.A[r, :]/T.A[r, s]
    for i = 1:size(T.A, 1)
        if i != r
            T.A[i, :] .= T.A[i, :] .- (T.A[i, s] * T.A[r, :])
        end
    end
    return T
end


function simplex_iteration(T::Tableau)
    
    k = 0
    #Can't increase z
    while (maximum(T.c) > 0) & (k < 100)
        #Column to pivot
        s = argmax(T.c)
        
        #Check if is unbounded
        if maximum(T.A[:, s]) <= 0
            error("Problem unbounded.")
        end

        #Ratio test to chose row to pivot
        r = 1
        min_ratio = T.b[1]/T.A[1, s]
        for i = 2:size(T.A, 1)
            if T.A[i, s] > 0
                new_ratio = T.b[i]/T.A[i, s]
                if new_ratio < min_ratio
                    r = i
                    min_ratio = new_ratio
                end
            end
        end
        
        T = pivot_tableau(T, r, s) 
        k = k +1
    end
    return T
end

simplex_iteration (generic function with 1 method)

In [8]:
function model_setup(c::Array{Float64,1},
                    A_ub::Array{Float64,2},
                    b_ub::Array{Float64,1},
                    A_eq::Array{Float64,2},
                    b_eq::Array{Float64,1},
                    verbose::Bool)
   """
    Function for linear programming using simplex algorithm
    Optimize the following problem
    
    Max:
        c * x
    subject to:
        A_ub * x <= b_ub
        A_eq * x == b_eq
    
    A_ub and A_eq should have the same number of columns as c (the number of variables)
    b_ub should have the same number of rows of A_ub and b_eq should have the same number of rows of A_eq
    
    TODO: real variables
    """ 
    
    n_variables = size(A_ub, 2)
    decisions_index = collect((1:n_variables))
    n_ub_const = size(A_ub, 1)
    n_eq_const = size(A_eq, 1)
    if verbose
        println("Total of " * string(n_ub_const) * " inequality constraints.")
        println("Total of " * string(n_eq_const) * " equality constraints.")
        println("Total of " * string(n_variables) * " variables.")
    end
    
    #adding slack variables
    A_ub = [A_ub Matrix(I(n_ub_const)*1.0)]
    A_eq = [A_eq zeros(size(A_eq, 1), n_ub_const)]
    slacks_index = collect((n_variables + 1):(n_variables + n_ub_const))
    c = [c; zeros(n_ub_const)]
    if verbose
        println("Added " * string(size(slacks_index)[1]) * " slack variables.")
    end
    
    basis = Dict{Int64,Int64}()
    #will add slack variables where b[i] >= 0 to the basis 
    for i in 1:n_ub_const
        if b_ub[i] >= 0
            push!(basis, i => (i+n_variables))
        #if is b[i] < 0, multiply row by -1
        else
            b_ub[i] = -1* b_ub[i]  
            A_ub[i, :] = -1 * A_ub[i, :]
        end
    end
    
    if verbose
        println("Slacks variables on the basis:")
        println(basis)
    end
    
    
    for i in 1:n_eq_const
        if b_eq[i] < 0
            b_eq[i] = -1*b_eq[i]
            A_eq[i, :] = -1 * A_eq[i, :]
        end
    end
    
    A = [A_ub; A_eq]
    b = [b_ub; b_eq; 0.0]
    n_const = size(A, 1)
    
    
    artificial_variables = Int64[]
    new_c = zeros(n_variables)
    new_b = zeros((n_const + 2))
    new_b[1:(n_const +1)] = b
    #for each row without a basis
    for i in 1:n_const
        if !(i in keys(basis))
            #row dont have basis
            #looking for isolated variable
            need_artificial = true
            for j in 1:size(A, 2)
                #check if there is isolated variable and not in basis
                if (sum(abs.(A[:, j])) == abs(A[i, j])) & !(j in values(basis))
                    #normalize row and add to basis
                    A[:, j] = A[:, j]/A[i, j]
                    push!(basis, i => j)
                    need_artificial = false
                    break
                end
            end
            
            if need_artificial
                aux = zeros(n_const)
                aux[i] = 1.0
                A = [A aux]
                artificial_variables = [artificial_variables; size(A, 2)]
                push!(basis, i => last(artificial_variables))
                new_c = new_c + A[i, 1:n_variables]
                new_b[size(new_b, 1)] = new_b[size(new_b, 1)] + b[i] 
            end
        end
    end
    
    if verbose
        println("Added " * string(size(artificial_variables, 1)) * " artificial variables.")
        println(basis)     
    end
    for i in artificial_variables
        new_c = [new_c; 0.0]
    end
    
    
    #if phase I is needed, otherwise go to phase II
    if size(artificial_variables, 1) > 0
        #add zeros for artificial variables
        for i in artificial_variables
            c = [c; 0.0]
        end
        
        #solve problem with artificial variables
        T = TableauI(A, new_b, c, new_c, basis)
        T = simplex_iterationI(T)
        if verbose
            println("Basis after phase I")
            println(T.basis)
        end
        
        #check if solution is equal to 0
        if last(T.b) == 0.0
            
            #check if there is artificial variable on basis
            for (key, value) in T.basis
                if value in artificial_variables
                    #check if every other value in this row is 0
                    non_zero_index = 0
                    all_zero = true
                    for j in 1:size(A, 2)
                        if T.A[key, j] != 0
                            all_zero = false
                            non_zero_index = copy(j)
                            break
                        end
                    end
                    
                    #don't have all zero
                    if !all_zero
                        #pivot on non zero column
                        T = pivot_tableauI(T, key, non_zero_index)
                    end
                end
            end
            
            if verbose
                println("Basis after removing artificial variables.")
                println(T.basis)
                println("Removing artificial variables of tableau")
            end
            for var in reverse!(artificial_variables)
                if !(var in values(T.basis))
                    T.A = T.A[:, 1:end .!=var]
                    T.c = T.c[1:end .!=var]
                end
            end
            T = Tableau(T.A, T.b[1:(size(T.A, 1) + 1)], T.c, T.basis)
            T =  simplex_iteration(T)
            return obtain_solution(T, decisions_index, slacks_index, artificial_variables)
        else
            error("Cant place in canonical form.")
        end
    else        
        T = Tableau(A, b, c, basis)
        T = simplex_iteration(T)
        return obtain_solution(T, decisions_index, slacks_index, artificial_variables)
    end
end

model_setup (generic function with 1 method)

In [29]:
A_ub = [[0.5 2 1];
    [1 2 4]]
b_ub = [24.0, 60.0];
c = [6.0, 14, 13];
A_eq = Array{Float64}(undef, 0, 3);
b_eq = Array{Float64}(undef,0 );
solution = model_setup(c, A_ub, b_ub, A_eq, b_eq, true)
println("Solution value" * string(solution.F))
println("Decision varaibles" * string(solution.Decision))

Total of 2 inequality constraints.
Total of 0 equality constraints.
Total of 3 variables.
Added 2 slack variables.
Slacks variables on the basis:
Dict(2 => 5,1 => 4)
Added 0 artificial variables.
Dict(2 => 5,1 => 4)
Solution value294.0
Decision varaibles[35.99999999999999, 0.0, 6.000000000000002]


In [30]:
A_eq = [[1.0 1 1 1];
    [-2 1 -1 0];
    [0 3 1 1]]
b_eq = [4.0, 1, 9]
c = [-3.0, 0, 1, 0]
A_ub = Array{Float64}(undef,0, 4);
b_ub = Array{Float64}(undef,0 );
solution = model_setup(c, A_ub, b_ub, A_eq, b_eq, true)
println("Solution value" * string(solution.F))
println("Decision varaibles" * string(solution.Decision))

Total of 0 inequality constraints.
Total of 3 equality constraints.
Total of 4 variables.
Added 0 slack variables.
Slacks variables on the basis:
Dict{Int64,Int64}()
Added 3 artificial variables.
Dict(2 => 6,3 => 7,1 => 5)
Basis after phase I
Dict(2 => 2,3 => 7,1 => 1)
Basis after removing artificial variables.
Dict(2 => 2,3 => 4,1 => 1)
Removing artificial variables of tableau
Solution value1.5
Decision varaibles[0.0, 2.5, 1.5, 0.0]


In [10]:
A_eq = [[1.0 -2 1];
    [-1 -3 1];
    [2 -3 4]]
b_eq = [2., 1, 7]
c = [1., 1, 1]
A_ub = Array{Float64}(undef,0, 3);
b_ub = Array{Float64}(undef,0 );
solution = model_setup(c, A_ub, b_ub, A_eq, b_eq, true)
println("Solution value" * string(solution.F))
println("Decision varaibles" * string(solution.Decision))

Total of 0 inequality constraints.
Total of 3 equality constraints.
Total of 3 variables.
Added 0 slack variables.
Slacks variables on the basis:
Dict{Int64,Int64}()
Added 3 artificial variables.
Dict(2 => 5,3 => 6,1 => 4)
Basis after phase I
Dict(2 => 3,3 => 2,1 => 1)
Basis after removing artificial variables.
Dict(2 => 3,3 => 2,1 => 1)
Removing artificial variables of tableau
Solution value1.9999999999999998
Decision varaibles[0.5, 0.0, 1.5]


In [9]:
#example with many variables and many constraints
function run_random(constraints::Int64, variables::Int64)
    A_ub = rand(Float64, (constraints, variables))
    b_ub = rand(Float64, (constraints)) * 5
    c = rand(Float64, (variables)) * 2
    A_eq = Array{Float64}(undef, 0, variables);
    b_eq = Array{Float64}(undef,0 );
    solution = model_setup(c, A_ub, b_ub, A_eq, b_eq, false)
end

run_random (generic function with 1 method)

In [10]:
using BenchmarkTools

In [11]:
@btime run_random(10, 15)

  6.899 μs (87 allocations: 20.58 KiB)


Solution(3.385906330637005, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6969998360021734, 0.0, 0.0, 0.0, 0.2864524501299704], [1.12955510611867, 1.0078762057259294, 0.0, 2.504973257659232, 1.2767773570186751, 2.769370374586511, 3.367944271060683, 3.1043088090155644, 0.0, 3.942078959554846])

In [12]:
@btime run_random(100, 15)

  89.400 μs (366 allocations: 587.77 KiB)


Solution(0.20856489141130147, [0.0, 0.0, 0.033757304932796564, 0.0, 0.0, 0.1992365900170919, 0.0, 0.0, 0.03547465944631663, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.4916716926596971, 1.3565420046762247, 4.298171899146448, 2.7581307584404, 3.3994654586570547, 1.5582384408586134, 3.17991266259131, 4.717706603958671, 2.6901621253458408, 4.3734997135772895  …  1.8534954703694564, 3.0681202142267647, 0.11483820606027347, 0.1339661871207433, 0.8852151775679284, 4.777364175745463, 2.691252133133439, 4.30409589957443, 3.1199098299659314, 0.5894495822848405])

In [18]:
@btime run_random(1000, 15)

  18.395 ms (3085 allocations: 47.06 MiB)


Solution(0.06221305037333735, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00027812236765778486, 0.0, 0.005502134035556994, 0.0, 0.0, 0.0, 0.0, 0.0, 0.029237514481134985], [1.0292238164531942, 1.574618274590502, 2.6150522516608765, 4.91523436653346, 3.867254286484046, 0.9314108187315226, 0.2925991765789897, 3.206725234625447, 4.162610838182865, 1.302442735261727  …  2.427523146886662, 2.346292501378106, 1.8760280656573434, 4.987176612029396, 0.15898425254934073, 3.7215429779679416, 4.065137158667132, 4.91699693463281, 4.227730002473541, 2.5658444063659713])

In [14]:
@btime run_random(10000, 15)

  36.334 s (420178 allocations: 17.93 GiB)


Solution(0.0012205370346504948, [0.0005586602089461733, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00026172568245654845, 0.0], [3.095859787343994, 3.445682611247458, 1.4404445110543955, 4.90303693548331, 3.747113034696484, 4.670563637145094, 2.454663652182882, 2.2521709173606435, 0.4408596056209593, 1.697590998597927  …  3.5052982526677186, 1.5570496488455217, 4.640174912772722, 2.8504033838457503, 4.648761674478382, 2.9516446135651253, 1.82079132680309, 3.7558614090140834, 4.8669282853464155, 2.7844942707894504])

In [15]:
@btime run_random(10, 20)

  7.200 μs (87 allocations: 23.47 KiB)


Solution(2.0082696295200693, [0.0, 0.0, 0.0, 0.0, 0.0, 0.7972796906418358, 0.0, 0.0, 0.0, 0.0, 0.5952498340863022, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.000621532414846, 0.831710530517439, 0.0, 1.2900949653380236, 2.9845283309194652, 3.8146214203069055, 2.9428465156462416, 0.0, 1.1187263452731875, 1.9941204336384692])

In [16]:
@btime run_random(100, 20)

  94.601 μs (366 allocations: 623.22 KiB)


Solution(-1.0407534216273089, [-1.8187439238006442, 0.0, 0.0, 0.0, 0.0, 0.0, 5.069842398956855, 0.0, -4.000661712931615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 5.166537723355261, -1.3460568367781556, 0.9973084321707759, 1.0819229347485777, 4.835515058603869, 4.53228991479525, 5.537330050056483, 2.9451180513561463, 3.3886156596891404  …  4.385674955222662, 1.3684618669830912, 4.052282619370095, 5.221501539969754, -2.1957853180895333, 1.3928211527576746, 0.8690839786331481, 4.24459279975762, -1.6188562494941938, 5.637089685201149])

In [17]:
@btime run_random(1000, 20)

  41.251 ms (6091 allocations: 71.21 MiB)


Solution(-4.001471945795533, [0.0, 8.469418654897245, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -16.06406034894548, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 5.363374391930585, 0.7823374875467106, 0.897296838858364, 7.0448923023755, -5.599246390061219, 3.7526998123916075, 4.271039393562813, 10.690951374950483, 1.895730927704753  …  9.693181573607514, 5.375992709254608, 9.488886839884225, 13.126766145030178, 5.25880760709268, 4.937901557245969, -0.05989775656112428, 1.3645778650004647, 5.317831950245043, 7.471128425456778])

In [ ]:
@btime run_random(10000, 20)